In [ ]:
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.2.2'
# spark_version = 'spark-3.<version number>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
# Allows for us to do operations with pyspark data frames and connect to postgres

!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

In [ ]:
# Create a new spark session
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
# Load the tsv file using SparkFiles and the tsv url
from pyspark import SparkFiles

url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Grocery_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Grocery_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="yyyy-mm-dd")

df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   42521656|R26MV8D0KG6QI6|B000SAQCWC|     159713740|The Cravings Plac...|         Grocery|          5|            0|          0|   N|                Y|Using these for y...|As a family aller...|2015-01-31 00:08:00|
|         US|   12049833|R1OF8GP57AQ1A0|B00509LVIQ|     138680402|Mauna Loa Macadam...| 

In [ ]:
# Drop null values
df = df.dropna()

# Count the number of records (rows) in the dataset
rows = df.count()
print(f"DataFrame Rows count: {rows}")

DataFrame Rows count: 2402212


In [ ]:
# Print schema
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



In [ ]:
# Create the review_id_table
review_id_table = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])

# Drop duplicate reviews
review_id_table = review_id_table.dropDuplicates()
review_id_table.show(3)

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
|R100GUW0J5VD4G|   22816749|B00DSQZNF0|     615254651|2015-01-16 00:06:00|
|R100VGJCANRF8E|   31402527|B001GVIT6E|      41991887|2013-01-06 00:11:00|
|R101085SG18YDI|    9818229|B006TH3XXY|     727501131|2015-01-04 00:06:00|
+--------------+-----------+----------+--------------+-------------------+
only showing top 3 rows



In [ ]:
# Create the products table
products = df.select(["product_id", "product_title"])

# Drop duplicate products
products = products.dropDuplicates()
products.show(3)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B0088PCJ1C|Bob's Pickle Pops...|
|B005P0U5BO|Envirokidz Organi...|
|B00K2RY8GI|Coffee Variety Sa...|
+----------+--------------------+
only showing top 3 rows



In [ ]:
# Create the customers table
customers = df.groupby("customer_id").agg({"customer_id":"count"})
customers = customers.withColumnRenamed("count(customer_id)", "customer_count")

# Check the schema of the customers table
customers.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [ ]:
# Cast customer_count column as an integer
from pyspark.sql.types import IntegerType
customers = customers.withColumn("customer_count", customers.customer_count.cast('integer'))

# Re-check the schema of the customers table
customers.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = false)



In [ ]:
# Reorder customer table by customer_count
from pyspark.sql.functions import desc
customers = customers.orderBy(desc("customer_count"))

customers.show(3)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   34247947|          1206|
|   17407953|           893|
|   12201275|           737|
+-----------+--------------+
only showing top 3 rows



In [ ]:
# Create the vine_table
vine_table = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])

vine_table.show(3)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R26MV8D0KG6QI6|          5|            0|          0|   N|
|R1OF8GP57AQ1A0|          5|            0|          0|   N|
|R3VDC1QB6MC4ZZ|          5|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 3 rows



In [ ]:
# Configure settings for RDS
mode = "append"
#jdbc_url="jdbc:postgresql://<connection string>:5432/<database-name>"
jdbc_url="jdbc:postgresql://:5432/groceriesdb"
config = {"user":"postgres", 
          "password": "", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to review_id_table table in RDS

review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write DataFrame to products table in RDS

products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write DataFrame to customers table in RDS

customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write DataFrame to vine_table table in RDS

vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)